# 1. Úkoly

## 1.1 Profilování a benchmarking
- Pomocí `line_profiler` zjistěte, které části kódu jsou nejnáročnější (pro `eps = 1e-100`).
- Proveďte benchmarking pomocí `time.time` pro `archimedes_pi` a `newton_pi` pro více hodnot `eps`:
  `1e-1, 1e-2, 1e-3, ..., 1e-100`.
- Výsledky zobrazte v log-log grafu.

In [ ]:
from decimal import Decimal as D, getcontext

getcontext().prec = 1000
pi = D("3.141592653589793238462643383279\
5028841971693993\
7510582097494459230781640628620899862803482534\
2117067982148086513282306647093844609550582231\
7253594081284811174502841027019385211055596446\
2294895493038196442881097566593344612847564823\
3786783165271201909145648566923460348610454326\
6482133936072602491412737245870066063155881748\
8152092096282925409171536436789259036001133053\
0548820466521384146951941511609433057270365759\
5919530921861173819326117931051185480744623799\
6274956735188575272489122793818301194912983367\
3362440656643086021394946395224737190702179860\
9437027705392171762931767523846748184676694051\
3200056812714526356082778577134275778960917363\
7178721468440901224953430146549585371050792279\
6892589235420199561121290219608640344181598136\
2977477130996051870721134999999837297804995105\
9731732816096318595024459455346908302642522308\
2533446850352619311881710100031378387528865875\
3320838142061717766914730359825349042875546873\
1159562863882353787593751957781857780532171226\
806613001927876611195909216420199")

def odmocnina(a, eps):
    x = a
    while abs(x**2 - a) > eps**2:
        x = (a / x + x) / 2
    return x


def archimedes_pi(eps):
    vyska = odmocnina(D(1) - (D(1) / D(2))**2, eps)
    zakladna = D(1)
    i = 1
    while True:
        zakladna = odmocnina((zakladna / 2)**2 + (1 - vyska)**2, eps)
        vyska = odmocnina(1 - (zakladna / 2)**2, eps)
        odhad_pi = 6 * (2**i) * zakladna * (vyska / 2)
        i += 1
        if abs(odhad_pi - pi) < eps:
            break
    return odhad_pi


def newton_pi(eps):
    a = D(0.5) * (D(1) / (2**3))
    pi_suma = D(1 / D(2 * 1 + 1)) * a
    pi_zbytek = ((-1) * odmocnina(D(3), eps)) / D(8) + D(1) / D(2)
    i = D(2)
    while True:
        a *= (D(2 * i - 3) / D(2 * i)) * D(1 / 4)
        pi_suma += (D(1) / D(2 * i + 1)) * a
        odhad_pi = 12 * (pi_zbytek - pi_suma)
        if abs(odhad_pi - pi) < eps:
            break
        i += 1
    return odhad_pi

eps = 1e-100
pi_odhad1 = archimedes_pi(eps)
pi_odhad2 = newton_pi(eps)

print(f"Pi podle Archimedova algoritmu s chybou {abs(pi - pi_odhad1):.4e}")
print(f"Pi podle Newtonova algoritmu s chybou {abs(pi - pi_odhad2):.4e}")

## 1.2 Prvočísla

Postup zopakujte pro funkce hledající prvočísla menší než `n`:
- profilováním najděte nejnáročnější části,
- benchmarkujte `eratosthenes_sieve` a `primes_test` pro `n = 10, 100, 1000, ..., 1_000_000`,
- výsledky zobrazte v log-log grafu.

In [ ]:
import numpy as np

def eratosthenes_sieve(n):
    sieve = np.arange(1, n + 1)
    sieve[0] = 0
    for i in range(2, int(np.sqrt(n)) + 1):
        if sieve[i - 1] != 0:
            sieve[i**2 - 1:n:i] = 0
    return np.where(sieve)[0] + 1

def primes_test(n):
    primes = [2]
    i = 3
    while i < n:
        for p in primes:
            if i % p == 0:
                break
            if p > np.sqrt(i):
                primes.append(i)
                break
        i += 2
    return np.array(primes)

n = 1000
print(eratosthenes_sieve(n))
print(primes_test(n))

## 1.3 Numba

1. Zrychlete obě funkce pro prvočísla pomocí Numby.
2. Porovnejte časy s původními verzemi a proveďte stejný benchmark jako výše.

## 1.4 Numba.stencil

2. Použijte `Numba.stencil` pro aproximaci difuze 2D pole v čase.
- Difuzní kernel má vycházet ze změn podle okolních buněk:
$$u_{i,j} = u_{i,j} + 
\frac{\lambda}{\delta_t}\sum_{k,l \in \{(i,j-1), (i-1,j), \ldots\}} u_{k,l} - u_{i,j}$$

In [ ]:
import numpy as np
from numba import stencil, jit
import matplotlib.pyplot as plt


lam = 1e-1
delta_t = 1

# Initialize the grid
N = 100  # Size of the grid (NxN)
grid = np.zeros((N, N))
grid[N // 2, N // 2] = 1000  # Initial concentration in the center

# TODO: naimplementujte funkci, která provede jeden krok difuze

def diffuse_step(grid):
    pass

In [ ]:
def diffuse(grid, num_steps):
    for time_step in range(num_steps):
        next_grid = diffuse_step(grid)
        if next_grid is None:
            raise NotImplementedError(
                'Funkce diffuse_step není implementovaná (záměrná výuková část úkolu).'
            )

        grid = next_grid
        if time_step % 100 == 0:
            plt.imshow(grid)
            plt.colorbar()
            plt.title(f'Diffusion after {time_step} time steps')
            plt.show()
            plt.close()
    return grid


# Simulate diffusion
num_steps = 1000
result = diffuse(grid, num_steps)